# Automated Machine Learning (AutoML) Search

## Background

### Machine Learning

[Machine learning](https://en.wikipedia.org/wiki/Machine_learning) (ML) is the process of constructing a mathematical model of a system based on a sample dataset collected from that system.

One of the main goals of training an ML model is to teach the model to separate the signal present in the data from the noise inherent in system and in the data collection process. If this is done effectively, the model can then be used to make accurate predictions about the system when presented with new, similar data. Additionally, introspecting on an ML model can reveal key information about the system being modeled, such as which inputs and transformations of the inputs are most useful to the ML model for learning the signal in the data, and are therefore the most predictive.

There are [a variety](https://en.wikipedia.org/wiki/Machine_learning#Approaches) of ML problem types. Supervised learning describes the case where the collected data contains an output value to be modeled and a set of inputs with which to train the model. EvalML focuses on training supervised learning models.

EvalML supports three common supervised ML problem types. The first is regression, where the target value to model is a continuous numeric value. Next are binary and multiclass classification, where the target value to model consists of two or more discrete values or categories. The choice of which supervised ML problem type is most appropriate depends on domain expertise and on how the model will be evaluated and used. 

EvalML is currently building support for supervised time series problems: time series regression, time series binary classification, and time series multiclass classification. While we've added some features to tackle these kinds of problems, our functionality is still being actively developed so please be mindful of that before using it. 


### AutoML and Search

[AutoML](https://en.wikipedia.org/wiki/Automated_machine_learning) is the process of automating the construction, training and evaluation of ML models. Given a data and some configuration, AutoML searches for the most effective and accurate ML model or models to fit the dataset. During the search, AutoML will explore different combinations of model type, model parameters and model architecture.

An effective AutoML solution offers several advantages over constructing and tuning ML models by hand. AutoML can assist with many of the difficult aspects of ML, such as avoiding overfitting and underfitting, imbalanced data, detecting data leakage and other potential issues with the problem setup, and automatically applying best-practice data cleaning, feature engineering, feature selection and various modeling techniques. AutoML can also leverage search algorithms to optimally sweep the hyperparameter search space, resulting in model performance which would be difficult to achieve by manual training.

## AutoML in EvalML

EvalML supports all of the above and more.

In its simplest usage, the AutoML search interface requires only the input data, the target data and a `problem_type` specifying what kind of supervised ML problem to model.

** Graphing methods, like AutoMLSearch, on Jupyter Notebook and Jupyter Lab require [ipywidgets](https://ipywidgets.readthedocs.io/en/latest/user_install.html) to be installed.

** If graphing on Jupyter Lab, [jupyterlab-plotly](https://plotly.com/python/getting-started/#jupyterlab-support-python-35) required. To download this, make sure you have [npm](https://nodejs.org/en/download/) installed.

In [2]:
import evalml
from evalml.utils import infer_feature_types
X, y = evalml.demos.load_fraud(n_rows=250)

             Number of Features
Boolean                       1
Categorical                   6
Numeric                       5

Number of training examples: 250
Targets
False    88.40%
True     11.60%
Name: fraud, dtype: object


To provide data to EvalML, it is recommended that you initialize a [Woodwork accessor](https://woodwork.alteryx.com/en/stable/) on your data. This allows you to easily control how EvalML will treat each of your features before training a model.

EvalML also accepts ``pandas`` input, and will run type inference on top of the input ``pandas`` data. If you'd like to change the types inferred by EvalML, you can use the `infer_feature_types` utility method, which takes pandas or numpy input and converts it to a Woodwork data structure. The `feature_types` parameter can be used to specify what types specific columns should be.

Feature types such as `Natural Language` must be specified in this way, otherwise Woodwork will infer it as `Unknown` type and drop it during the AutoMLSearch.

In the example below, we reformat a couple features to make them easily consumable by the model, and then specify that the provider, which would have otherwise been inferred as a column with natural language, is a categorical column.

In [3]:
X.ww['expiration_date'] = X['expiration_date'].apply(lambda x: '20{}-01-{}'.format(x.split("/")[1], x.split("/")[0]))
X = infer_feature_types(X, feature_types= {'store_id': 'categorical',
                                           'expiration_date': 'datetime',
                                           'lat': 'categorical',
                                           'lng': 'categorical',
                                           'provider': 'categorical'})

In order to validate the results of the pipeline creation and optimization process, we will save some of our data as a holdout set.

In [4]:
X_train, X_holdout, y_train, y_holdout = evalml.preprocessing.split_data(X, y, problem_type='binary', test_size=.2)

### Data Checks

Before calling `AutoMLSearch.search`, we should run some sanity checks on our data to ensure that the input data being passed will not run into some common issues before running a potentially time-consuming search. EvalML has various data checks that makes this easy. Each data check will return a collection of warnings and errors if it detects potential issues with the input data. This allows users to inspect their data to avoid confusing errors that may arise during the search process. You can learn about each of the data checks available through our [data checks guide](data_checks.ipynb) 

Here, we will run the `DefaultDataChecks` class, which contains a series of data checks that are generally useful.

In [5]:
from evalml.data_checks import DefaultDataChecks

data_checks = DefaultDataChecks("binary", "log loss binary")
data_checks.validate(X_train, y_train)

{'warnings': [], 'errors': [], 'actions': []}

Since there were no warnings or errors returned, we can safely continue with the search process.

In [6]:
automl = evalml.automl.AutoMLSearch(X_train=X_train, y_train=y_train, problem_type='binary')
automl.search()

Using default limit of max_batches=1.

Generating pipelines to search over...
8 pipelines ready for search.

*****************************
* Beginning pipeline search *
*****************************

Optimizing for Log Loss Binary. 
Lower score is better.

Using SequentialEngine to train and score pipelines.
Searching up to 1 batches for a total of 9 pipelines. 
Allowed model families: extra_trees, lightgbm, linear_model, xgboost, random_forest, decision_tree, catboost



FigureWidget({
    'data': [{'mode': 'lines+markers',
              'name': 'Best Score',
              'type'…

Evaluating Baseline Pipeline: Mode Baseline Binary Classification Pipeline
Mode Baseline Binary Classification Pipeline:
	Starting cross validation
	Finished cross validation - mean Log Loss Binary: 3.970

*****************************
* Evaluating Batch Number 1 *
*****************************

Elastic Net Classifier w/ Imputer + DateTime Featurization Component + One Hot Encoder + SMOTENC Oversampler + Standard Scaler:
	Starting cross validation
	Finished cross validation - mean Log Loss Binary: 0.512
Decision Tree Classifier w/ Imputer + DateTime Featurization Component + One Hot Encoder + SMOTENC Oversampler:
	Starting cross validation
	Finished cross validation - mean Log Loss Binary: 2.957
	High coefficient of variation (cv >= 0.2) within cross validation scores.
	Decision Tree Classifier w/ Imputer + DateTime Featurization Component + One Hot Encoder + SMOTENC Oversampler may not perform as estimated on unseen data.
Random Forest Classifier w/ Imputer + DateTime Featurization Co

/usr/local/Caskroom/miniconda/base/envs/evalml_dev/lib/python3.9/site-packages/xgboost/sklearn.py:1146: UserWarning:

The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].



[14:55:37] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
			Fold 0: Encountered an error.
			Fold 0: All scores will be replaced with nan.
			Fold 0: Please check the log file for the current hyperparameters and stack trace.
			Fold 0: Exception during automl search: [14:55:38] ../src/c_api/../data/array_interface.h:139: Check failed: typestr.size() == 3 (2 vs. 3) : `typestr' should be of format <endian><type><size of type in bytes>.
Stack trace:
  [bt] (0) 1   libxgboost.dylib                    0x00000001a2b2b074 dmlc::LogMessageFatal::~LogMessageFatal() + 116
  [bt] (1) 2   libxgboost.dylib                    0x00000001a2b25773 xgboost::ArrayInterfaceHandler::Validate(std::__1::map<std::__1::basic_string<char, std::__1::char_traits<char>, std::__1::allocator<char> >, xgboost::Json, std::__1:

/usr/local/Caskroom/miniconda/base/envs/evalml_dev/lib/python3.9/site-packages/xgboost/data.py:112: UserWarning:

Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption

/usr/local/Caskroom/miniconda/base/envs/evalml_dev/lib/python3.9/site-packages/xgboost/sklearn.py:1146: UserWarning:

The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].

/usr/local/Caskroom/miniconda/base/envs/evalml_dev/lib/python3.9/site-packages/xgboost/data.py:112: UserWarning:

Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption



[14:55:38] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
			Fold 1: Encountered an error.
			Fold 1: All scores will be replaced with nan.
			Fold 1: Please check the log file for the current hyperparameters and stack trace.
			Fold 1: Exception during automl search: [14:55:38] ../src/c_api/../data/array_interface.h:139: Check failed: typestr.size() == 3 (2 vs. 3) : `typestr' should be of format <endian><type><size of type in bytes>.
Stack trace:
  [bt] (0) 1   libxgboost.dylib                    0x00000001a2b2b074 dmlc::LogMessageFatal::~LogMessageFatal() + 116
  [bt] (1) 2   libxgboost.dylib                    0x00000001a2b25773 xgboost::ArrayInterfaceHandler::Validate(std::__1::map<std::__1::basic_string<char, std::__1::char_traits<char>, std::__1::allocator<char> >, xgboost::Json, std::__1:

/usr/local/Caskroom/miniconda/base/envs/evalml_dev/lib/python3.9/site-packages/xgboost/sklearn.py:1146: UserWarning:

The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].

/usr/local/Caskroom/miniconda/base/envs/evalml_dev/lib/python3.9/site-packages/xgboost/data.py:112: UserWarning:

Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption



[14:55:38] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
			Fold 2: Encountered an error.
			Fold 2: All scores will be replaced with nan.
			Fold 2: Please check the log file for the current hyperparameters and stack trace.
			Fold 2: Exception during automl search: [14:55:39] ../src/c_api/../data/array_interface.h:139: Check failed: typestr.size() == 3 (2 vs. 3) : `typestr' should be of format <endian><type><size of type in bytes>.
Stack trace:
  [bt] (0) 1   libxgboost.dylib                    0x00000001a2b2b074 dmlc::LogMessageFatal::~LogMessageFatal() + 116
  [bt] (1) 2   libxgboost.dylib                    0x00000001a2b25773 xgboost::ArrayInterfaceHandler::Validate(std::__1::map<std::__1::basic_string<char, std::__1::char_traits<char>, std::__1::allocator<char> >, xgboost::Json, std::__1:

The AutoML search will log its progress, reporting each pipeline and parameter set evaluated during the search.

There are a number of mechanisms to control the AutoML search time. One way is to set the `max_batches` parameter which controls the maximum number of rounds of AutoML to evaluate, where each round may train and score a variable number of pipelines. Another way is to set the `max_iterations` parameter which controls the maximum number of candidate models to be evaluated during AutoML. By default, AutoML will search for a single batch. The first pipeline to be evaluated will always be a baseline model representing a trivial solution. 

The AutoML interface supports a variety of other parameters. For a comprehensive list, please [refer to the API reference.](../autoapi/evalml/automl/index.rst#evalml.automl.AutoMLSearch)

We also provide [a standalone search method](../autoapi/evalml/automl/index.rst#evalml.automl.search) which does all of the above in a single line, and returns the `AutoMLSearch` instance and data check results. If there were data check errors, AutoML will not be run and no `AutoMLSearch` instance will be returned.

### Detecting Problem Type

EvalML includes a simple method, `detect_problem_type`, to help determine the problem type given the target data. 

This function can return the predicted problem type as a ProblemType enum, choosing from ProblemType.BINARY, ProblemType.MULTICLASS, and ProblemType.REGRESSION. If the target data is invalid (for instance when there is only 1 unique label), the function will throw an error instead.


In [7]:
import pandas as pd
from evalml.problem_types import detect_problem_type

y_binary = pd.Series([0, 1, 1, 0, 1, 1])
detect_problem_type(y_binary)

<ProblemTypes.BINARY: 'binary'>

### Objective parameter

AutoMLSearch takes in an `objective` parameter to determine which `objective` to optimize for. By default, this parameter is set to `auto`, which allows AutoML to choose `LogLossBinary` for binary classification problems, `LogLossMulticlass` for multiclass classification problems, and `R2` for regression problems.

It should be noted that the `objective` parameter is only used in ranking and helping choose the pipelines to iterate over, but is not used to optimize each individual pipeline during fit-time.

To get the default objective for each problem type, you can use the `get_default_primary_search_objective` function.

In [8]:
from evalml.automl import get_default_primary_search_objective

binary_objective = get_default_primary_search_objective("binary")
multiclass_objective = get_default_primary_search_objective("multiclass")
regression_objective = get_default_primary_search_objective("regression")

print(binary_objective.name)
print(multiclass_objective.name)
print(regression_objective.name)

Log Loss Binary
Log Loss Multiclass
R2


### Using custom pipelines

EvalML's AutoML algorithm generates a set of pipelines to search with. To provide a custom set instead, set allowed_component_graphs to a dictionary of custom component graphs. `AutoMLSearch` will use these to generate `Pipeline` instances. Note: this will prevent AutoML from generating other pipelines to search over.

In [9]:
from evalml.pipelines import MulticlassClassificationPipeline


automl_custom = evalml.automl.AutoMLSearch(X_train=X_train,
                                           y_train=y_train,
                                           problem_type='multiclass',
                                           allowed_component_graphs={"My_pipeline": ['Simple Imputer', 'Random Forest Classifier'],
                                                                     "My_other_pipeline": ['One Hot Encoder', 'Random Forest Classifier']})

Using default limit of max_batches=1.

2 pipelines ready for search.


### Stopping the search early

To stop the search early, hit `Ctrl-C`. This will bring up a prompt asking for confirmation. Responding with `y` will immediately stop the search. Responding with `n` will continue the search.

![Interrupting Search Demo](keyboard_interrupt_demo_updated.gif)

### Callback functions

``AutoMLSearch`` supports several callback functions, which can be specified as parameters when initializing an ``AutoMLSearch`` object. They are:

- ``start_iteration_callback``
- ``add_result_callback``
- ``error_callback``


#### Start Iteration Callback
Users can set ``start_iteration_callback`` to set what function is called before each pipeline training iteration. This callback function must take three positional parameters: the pipeline class, the pipeline parameters, and the ``AutoMLSearch`` object.

In [10]:
## start_iteration_callback example function
def start_iteration_callback_example(pipeline_class, pipeline_params, automl_obj):
    print ("Training pipeline with the following parameters:", pipeline_params)

#### Add Result Callback
Users can set ``add_result_callback`` to set what function is called after each pipeline training iteration. This callback function must take three positional parameters: a dictionary containing the training results for the new pipeline, an untrained_pipeline containing the parameters used during training, and the ``AutoMLSearch`` object.

In [11]:
## add_result_callback example function
def add_result_callback_example(pipeline_results_dict, untrained_pipeline, automl_obj):
    print ("Results for trained pipeline with the following parameters:", pipeline_results_dict)

#### Error Callback
Users can set the ``error_callback`` to set what function called when `search()` errors and raises an ``Exception``. This callback function takes three positional parameters: the ``Exception raised``, the traceback, and the ``AutoMLSearch object``. This callback function must also accept ``kwargs``, so ``AutoMLSearch`` is able to pass along other parameters used by default.

Evalml defines several error callback functions, which can be found under `evalml.automl.callbacks`. They are:

- `silent_error_callback`
- `raise_error_callback`
- `log_and_save_error_callback`
- `raise_and_save_error_callback`
- `log_error_callback` (default used when ``error_callback`` is None)

In [12]:
# error_callback example; this is implemented in the evalml library
def raise_error_callback(exception, traceback, automl, **kwargs):
    """Raises the exception thrown by the AutoMLSearch object. Also logs the exception as an error."""
    logger.error(f'AutoMLSearch raised a fatal exception: {str(exception)}')
    logger.error("\n".join(traceback))
    raise exception

## View Rankings
A summary of all the pipelines built can be returned as a pandas DataFrame which is sorted by score. The score column contains the average score across all cross-validation folds while the validation_score column is computed from the first cross-validation fold.

In [13]:
automl.rankings

,id,pipeline_name,search_order,mean_cv_score,standard_deviation_cv_score,validation_score,percent_better_than_baseline,high_variance_cv,parameters
0,3,Random Forest Classifier w/ Imputer + DateTime...,3,0.285613,0.041116,0.263695,92.806475,False,{'Imputer': {'categorical_impute_strategy': 'm...
1,4,LightGBM Classifier w/ Imputer + DateTime Feat...,4,0.308636,0.203878,0.234947,92.226623,True,{'Imputer': {'categorical_impute_strategy': 'm...
2,7,Extra Trees Classifier w/ Imputer + DateTime F...,7,0.338286,0.009381,0.329015,91.479857,False,{'Imputer': {'categorical_impute_strategy': 'm...
3,1,Elastic Net Classifier w/ Imputer + DateTime F...,1,0.511808,0.074992,0.590517,87.109474,False,{'Imputer': {'categorical_impute_strategy': 'm...
4,5,Logistic Regression Classifier w/ Imputer + Da...,5,0.551960,0.082695,0.628646,86.098206,False,{'Imputer': {'categorical_impute_strategy': 'm...
5,8,CatBoost Classifier w/ Imputer + DateTime Feat...,8,0.601819,0.007246,0.593693,84.842444,False,{'Imputer': {'categorical_impute_strategy': 'm...
6,2,Decision Tree Classifier w/ Imputer + DateTime...,2,2.956956,3.084439,0.651557,25.525413,True,{'Imputer': {'categorical_impute_strategy': 'm...
7,0,Mode Baseline Binary Classification Pipeline,0,3.970423,0.266060,4.124033,0.000000,False,{'Baseline Classifier': {'strategy': 'mode'}}
8,6,XGBoost Classifier w/ Imputer + DateTime Featu...,6,NaN,NaN,NaN,NaN,False,{'Imputer': {'categorical_impute_strategy': 'm...


## Describe Pipeline
Each pipeline is given an `id`. We can get more information about any particular pipeline using that `id`. Here, we will get more information about the pipeline with `id = 1`.

In [14]:
automl.describe_pipeline(1)


**********************************************************************************************************************************
* Elastic Net Classifier w/ Imputer + DateTime Featurization Component + One Hot Encoder + SMOTENC Oversampler + Standard Scaler *
**********************************************************************************************************************************

Problem Type: binary
Model Family: Linear

Pipeline Steps
1. Imputer
	 * categorical_impute_strategy : most_frequent
	 * numeric_impute_strategy : mean
	 * categorical_fill_value : None
	 * numeric_fill_value : None
2. DateTime Featurization Component
	 * features_to_extract : ['year', 'month', 'day_of_week', 'hour']
	 * encode_as_categories : False
	 * date_index : None
3. One Hot Encoder
	 * top_n : 10
	 * features_to_encode : None
	 * categories : None
	 * drop : if_binary
	 * handle_unknown : ignore
	 * handle_missing : error
4. SMOTENC Oversampler
	 * sampling_ratio : 0.25
	 * k_neighbors_defa

## Get Pipeline
We can get the object of any pipeline via their `id` as well:

In [15]:
pipeline = automl.get_pipeline(1)
print(pipeline.name)
print(pipeline.parameters)

Elastic Net Classifier w/ Imputer + DateTime Featurization Component + One Hot Encoder + SMOTENC Oversampler + Standard Scaler
{'Imputer': {'categorical_impute_strategy': 'most_frequent', 'numeric_impute_strategy': 'mean', 'categorical_fill_value': None, 'numeric_fill_value': None}, 'DateTime Featurization Component': {'features_to_extract': ['year', 'month', 'day_of_week', 'hour'], 'encode_as_categories': False, 'date_index': None}, 'One Hot Encoder': {'top_n': 10, 'features_to_encode': None, 'categories': None, 'drop': 'if_binary', 'handle_unknown': 'ignore', 'handle_missing': 'error'}, 'SMOTENC Oversampler': {'sampling_ratio': 0.25, 'k_neighbors_default': 5, 'n_jobs': -1, 'sampling_ratio_dict': None, 'categorical_features': [3, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59], 'k_neighbors': 5}, 'Elastic Net Classifier': {'penalty': '

### Get best pipeline
If you specifically want to get the best pipeline, there is a convenient accessor for that.
The pipeline returned is already fitted on the input X, y data that we passed to AutoMLSearch. To turn off this default behavior, set `train_best_pipeline=False` when initializing AutoMLSearch.

In [16]:
best_pipeline = automl.best_pipeline
print(best_pipeline.name)
print(best_pipeline.parameters)
best_pipeline.predict(X_train)

Random Forest Classifier w/ Imputer + DateTime Featurization Component + One Hot Encoder + SMOTENC Oversampler
{'Imputer': {'categorical_impute_strategy': 'most_frequent', 'numeric_impute_strategy': 'mean', 'categorical_fill_value': None, 'numeric_fill_value': None}, 'DateTime Featurization Component': {'features_to_extract': ['year', 'month', 'day_of_week', 'hour'], 'encode_as_categories': False, 'date_index': None}, 'One Hot Encoder': {'top_n': 10, 'features_to_encode': None, 'categories': None, 'drop': 'if_binary', 'handle_unknown': 'ignore', 'handle_missing': 'error'}, 'SMOTENC Oversampler': {'sampling_ratio': 0.25, 'k_neighbors_default': 5, 'n_jobs': -1, 'sampling_ratio_dict': None, 'categorical_features': [3, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59], 'k_neighbors': 5}, 'Random Forest Classifier': {'n_estimators': 100, 'max_

0      False
1      False
2      False
3      False
4      False
       ...  
195    False
196    False
197    False
198    False
199    False
Name: fraud, Length: 200, dtype: bool

## Training and Scoring Multiple Pipelines using AutoMLSearch

AutoMLSearch will automatically fit the best pipeline on the entire training data. It also provides an easy API for training and scoring other pipelines.

If you'd like to train one or more pipelines on the entire training data, you can use the `train_pipelines`method

Similarly, if you'd like to score one or more pipelines on a particular dataset, you can use the `train_pipelines`method


In [17]:
trained_pipelines = automl.train_pipelines([automl.get_pipeline(i) for i in [0, 1, 2]])
trained_pipelines


{'Mode Baseline Binary Classification Pipeline': pipeline = BinaryClassificationPipeline(component_graph={'Baseline Classifier': ['Baseline Classifier', 'X', 'y']}, parameters={'Baseline Classifier':{'strategy': 'mode'}}, custom_name='Mode Baseline Binary Classification Pipeline', random_seed=0),
 'Elastic Net Classifier w/ Imputer + DateTime Featurization Component + One Hot Encoder + SMOTENC Oversampler + Standard Scaler': pipeline = BinaryClassificationPipeline(component_graph={'Imputer': ['Imputer', 'X', 'y'], 'DateTime Featurization Component': ['DateTime Featurization Component', 'Imputer.x', 'y'], 'One Hot Encoder': ['One Hot Encoder', 'DateTime Featurization Component.x', 'y'], 'SMOTENC Oversampler': ['SMOTENC Oversampler', 'One Hot Encoder.x', 'y'], 'Standard Scaler': ['Standard Scaler', 'SMOTENC Oversampler.x', 'SMOTENC Oversampler.y'], 'Elastic Net Classifier': ['Elastic Net Classifier', 'Standard Scaler.x', 'SMOTENC Oversampler.y']}, parameters={'Imputer':{'categorical_impu

In [18]:
pipeline_holdout_scores = automl.score_pipelines([trained_pipelines[name] for name in trained_pipelines.keys()],
                                                X_holdout,
                                                y_holdout,
                                                ['Accuracy Binary', 'F1', 'AUC'])
pipeline_holdout_scores


{'Mode Baseline Binary Classification Pipeline': OrderedDict([('Accuracy Binary',
               0.88),
              ('F1', 0.0),
              ('AUC', 0.5)]),
 'Elastic Net Classifier w/ Imputer + DateTime Featurization Component + One Hot Encoder + SMOTENC Oversampler + Standard Scaler': OrderedDict([('Accuracy Binary',
               0.66),
              ('F1', 0.19047619047619044),
              ('AUC', 0.5265151515151515)]),
 'Decision Tree Classifier w/ Imputer + DateTime Featurization Component + One Hot Encoder + SMOTENC Oversampler': OrderedDict([('Accuracy Binary',
               0.92),
              ('F1', 0.6),
              ('AUC', 0.7234848484848486)])}

## Saving AutoMLSearch and pipelines from AutoMLSearch

There are two ways to save results from AutoMLSearch. 

- You can save the AutoMLSearch object itself, calling `.save(<filepath>)` to do so. This will allow you to save the AutoMLSearch state and reload all pipelines from this.

- If you want to save a pipeline from AutoMLSearch for future use, pipeline classes themselves have a `.save(<filepath>)` method.

In [19]:
# saving the entire automl search
automl.save("automl.cloudpickle")
automl2 = evalml.automl.AutoMLSearch.load("automl.cloudpickle")
# saving the best pipeline using .save()
best_pipeline.save("pipeline.cloudpickle")
best_pipeline_copy = evalml.pipelines.PipelineBase.load("pipeline.cloudpickle")

## Limiting the AutoML Search Space
The AutoML search algorithm first trains each component in the pipeline with their default values. After the first iteration, it then tweaks the parameters of these components using the pre-defined hyperparameter ranges that these components have. To limit the search over certain hyperparameter ranges, you can specify a `custom_hyperparameters` argument with your `AutoMLSearch` parameters. These parameters will limit the hyperparameter search space. 

Hyperparameter ranges can be found through the [API reference](https://evalml.alteryx.com/en/stable/api_reference.html) for each component. Parameter arguments must be specified as dictionaries, but the associated values can be single values or `skopt.space` Real, Integer, Categorical values.

If however you'd like to specify certain values for the initial batch of the AutoML search algorithm, you can use the `pipeline_parameters` argument. This will set the initial batch's component parameters to the values passed by this argument.

In [20]:
from evalml import AutoMLSearch
from evalml.demos import load_fraud
from skopt.space import Categorical
from evalml.model_family import ModelFamily
import woodwork as ww

X, y = load_fraud(n_rows=1000)

# example of setting parameter to just one value
custom_hyperparameters = {'Imputer': {
    'numeric_impute_strategy': 'mean'
}}


# limit the numeric impute strategy to include only `median` and `most_frequent`
# `mean` is the default value for this argument, but it doesn't need to be included in the specified hyperparameter range for this to work
custom_hyperparameters = {'Imputer': {
    'numeric_impute_strategy': Categorical(['median', 'most_frequent'])
}}
# set the initial batch numeric impute strategy strategy to 'median'
pipeline_parameters = {'Imputer': {
    'numeric_impute_strategy': 'median'
}}

# using this custom hyperparameter means that our Imputer components in these pipelines will only search through
# 'median' and 'most_frequent' strategies for 'numeric_impute_strategy', and the initial batch parameter will be
# set to 'median'
automl_constrained = AutoMLSearch(X_train=X, y_train=y, problem_type='binary', pipeline_parameters=pipeline_parameters,
                                  custom_hyperparameters=custom_hyperparameters)

             Number of Features
Boolean                       1
Categorical                   6
Numeric                       5

Number of training examples: 1000
Targets
False    85.90%
True     14.10%
Name: fraud, dtype: object
Using default limit of max_batches=1.

Generating pipelines to search over...
8 pipelines ready for search.


## Imbalanced Data
The AutoML search algorithm now has functionality to handle imbalanced data during classification! AutoMLSearch now provides two additional parameters, `sampler_method` and `sampler_balanced_ratio`, that allow you to let AutoMLSearch know whether to sample imbalanced data, and how to do so. `sampler_method` takes in either `Undersampler`, `Oversampler`, `auto`, or None as the sampler to use, and `sampler_balanced_ratio` specifies the `minority/majority` ratio that you want to sample to. Details on the Undersampler and Oversampler components can be found in the [documentation](https://evalml.alteryx.com/en/stable/api_reference.html#transformers).

This can be used for imbalanced datasets, like the fraud dataset, which has a 'minority:majority' ratio of < 0.2.

In [21]:
automl_auto = AutoMLSearch(X_train=X, y_train=y, problem_type='binary')
automl_auto.allowed_pipelines[-1]

Using default limit of max_batches=1.

Generating pipelines to search over...
8 pipelines ready for search.


pipeline = BinaryClassificationPipeline(component_graph={'Imputer': ['Imputer', 'X', 'y'], 'DateTime Featurization Component': ['DateTime Featurization Component', 'Imputer.x', 'y'], 'One Hot Encoder': ['One Hot Encoder', 'DateTime Featurization Component.x', 'y'], 'SMOTENC Oversampler': ['SMOTENC Oversampler', 'One Hot Encoder.x', 'y'], 'Standard Scaler': ['Standard Scaler', 'SMOTENC Oversampler.x', 'SMOTENC Oversampler.y'], 'Logistic Regression Classifier': ['Logistic Regression Classifier', 'Standard Scaler.x', 'SMOTENC Oversampler.y']}, parameters={'Imputer':{'categorical_impute_strategy': 'most_frequent', 'numeric_impute_strategy': 'mean', 'categorical_fill_value': None, 'numeric_fill_value': None}, 'DateTime Featurization Component':{'features_to_extract': ['year', 'month', 'day_of_week', 'hour'], 'encode_as_categories': False, 'date_index': None}, 'One Hot Encoder':{'top_n': 10, 'features_to_encode': None, 'categories': None, 'drop': 'if_binary', 'handle_unknown': 'ignore', 'han

The SMOTENC Oversampler is chosen as the default sampling component here, since the `sampler_balanced_ratio = 0.25`. If you specified a lower ratio, for instance `sampler_balanced_ratio = 0.1`, then there would be no sampling component added here. This is because if a ratio of 0.1 would be considered balanced, then a ratio of 0.2 would also be balanced.

In [22]:
automl_auto_ratio = AutoMLSearch(X_train=X, y_train=y, problem_type='binary', sampler_balanced_ratio=0.1)
automl_auto_ratio.allowed_pipelines[-1]

Using default limit of max_batches=1.

Generating pipelines to search over...
8 pipelines ready for search.


pipeline = BinaryClassificationPipeline(component_graph={'Imputer': ['Imputer', 'X', 'y'], 'DateTime Featurization Component': ['DateTime Featurization Component', 'Imputer.x', 'y'], 'One Hot Encoder': ['One Hot Encoder', 'DateTime Featurization Component.x', 'y'], 'Standard Scaler': ['Standard Scaler', 'One Hot Encoder.x', 'y'], 'Logistic Regression Classifier': ['Logistic Regression Classifier', 'Standard Scaler.x', 'y']}, parameters={'Imputer':{'categorical_impute_strategy': 'most_frequent', 'numeric_impute_strategy': 'mean', 'categorical_fill_value': None, 'numeric_fill_value': None}, 'DateTime Featurization Component':{'features_to_extract': ['year', 'month', 'day_of_week', 'hour'], 'encode_as_categories': False, 'date_index': None}, 'One Hot Encoder':{'top_n': 10, 'features_to_encode': None, 'categories': None, 'drop': 'if_binary', 'handle_unknown': 'ignore', 'handle_missing': 'error'}, 'Logistic Regression Classifier':{'penalty': 'l2', 'C': 1.0, 'n_jobs': -1, 'multi_class': 'aut

Additionally, you can add more fine-grained sampling ratios by passing in a `sampling_ratio_dict` in pipeline parameters. For this dictionary, AutoMLSearch expects the keys to be int values from 0 to `n-1` for the classes, and the values would be the `sampler_balanced__ratio` associated with each target. This dictionary would override the AutoML argument `sampler_balanced_ratio`. Below, you can see the scenario for Oversampler component on this dataset. Note that the logic for Undersamplers is included in the commented section.

In [23]:
# In this case, the majority class is the negative class
# for the oversampler, we don't want to oversample this class, so class 0 (majority) will have a ratio of 1 to itself
# for the minority class 1, we want to oversample it to have a minority/majority ratio of 0.5, which means we want minority to have 1/2 the samples as the minority
sampler_ratio_dict = {0: 1, 1: 0.5}
pipeline_parameters = {"SMOTENC Oversampler": {"sampler_balanced_ratio": sampler_ratio_dict}}
automl_auto_ratio_dict = AutoMLSearch(X_train=X, y_train=y, problem_type='binary', pipeline_parameters=pipeline_parameters)
automl_auto_ratio_dict.allowed_pipelines[-1]

# Undersampler case
# we don't want to undersample this class, so class 1 (minority) will have a ratio of 1 to itself
# for the majority class 0, we want to undersample it to have a minority/majority ratio of 0.5, which means we want majority to have 2x the samples as the minority
# sampler_ratio_dict = {0: 0.5, 1: 1}
# pipeline_parameters = {"SMOTENC Oversampler": {"sampler_balanced_ratio": sampler_ratio_dict}}
# automl_auto_ratio_dict = AutoMLSearch(X_train=X, y_train=y, problem_type='binary', pipeline_parameters=pipeline_parameters)


Using default limit of max_batches=1.

Generating pipelines to search over...
8 pipelines ready for search.


pipeline = BinaryClassificationPipeline(component_graph={'Imputer': ['Imputer', 'X', 'y'], 'DateTime Featurization Component': ['DateTime Featurization Component', 'Imputer.x', 'y'], 'One Hot Encoder': ['One Hot Encoder', 'DateTime Featurization Component.x', 'y'], 'SMOTENC Oversampler': ['SMOTENC Oversampler', 'One Hot Encoder.x', 'y'], 'Standard Scaler': ['Standard Scaler', 'SMOTENC Oversampler.x', 'SMOTENC Oversampler.y'], 'Logistic Regression Classifier': ['Logistic Regression Classifier', 'Standard Scaler.x', 'SMOTENC Oversampler.y']}, parameters={'Imputer':{'categorical_impute_strategy': 'most_frequent', 'numeric_impute_strategy': 'mean', 'categorical_fill_value': None, 'numeric_fill_value': None}, 'DateTime Featurization Component':{'features_to_extract': ['year', 'month', 'day_of_week', 'hour'], 'encode_as_categories': False, 'date_index': None}, 'One Hot Encoder':{'top_n': 10, 'features_to_encode': None, 'categories': None, 'drop': 'if_binary', 'handle_unknown': 'ignore', 'han

## Adding ensemble methods to AutoML

### Stacking
[Stacking](https://en.wikipedia.org/wiki/Ensemble_learning#Stacking) is an ensemble machine learning algorithm that involves training a model to best combine the predictions of several base learning algorithms. First, each base learning algorithms is trained using the given data. Then, the combining algorithm or meta-learner is trained on the predictions made by those base learning algorithms to make a final prediction.

AutoML enables stacking using the `ensembling` flag during initalization; this is set to `False` by default. The stacking ensemble pipeline runs in its own batch after a whole cycle of training has occurred (each allowed pipeline trains for one batch). Note that this means __a large number of iterations may need to run before the stacking ensemble runs__. It is also important to note that __only the first CV fold is calculated for stacking ensembles__ because the model internally uses CV folds.

In [24]:
X, y = evalml.demos.load_breast_cancer()

automl_with_ensembling = AutoMLSearch(X_train=X, y_train=y,
                                      problem_type="binary",
                                      allowed_model_families=[ModelFamily.LINEAR_MODEL],
                                      max_batches=4,
                                      ensembling=True)
automl_with_ensembling.search()

         Number of Features
Numeric                  30

Number of training examples: 569
Targets
benign       62.74%
malignant    37.26%
Name: target, dtype: object
Generating pipelines to search over...
2 pipelines ready for search.
Ensembling will run every 3 batches.

*****************************
* Beginning pipeline search *
*****************************

Optimizing for Log Loss Binary. 
Lower score is better.

Using SequentialEngine to train and score pipelines.
Searching up to 4 batches for a total of 14 pipelines. 
Allowed model families: linear_model



FigureWidget({
    'data': [{'mode': 'lines+markers',
              'name': 'Best Score',
              'type'…

Evaluating Baseline Pipeline: Mode Baseline Binary Classification Pipeline
Mode Baseline Binary Classification Pipeline:
	Starting cross validation
	Finished cross validation - mean Log Loss Binary: 12.868

*****************************
* Evaluating Batch Number 1 *
*****************************

Elastic Net Classifier w/ Imputer + Standard Scaler:
	Starting cross validation
	Finished cross validation - mean Log Loss Binary: 0.077
Logistic Regression Classifier w/ Imputer + Standard Scaler:
	Starting cross validation
	Finished cross validation - mean Log Loss Binary: 0.077
	High coefficient of variation (cv >= 0.2) within cross validation scores.
	Logistic Regression Classifier w/ Imputer + Standard Scaler may not perform as estimated on unseen data.

*****************************
* Evaluating Batch Number 2 *
*****************************

Logistic Regression Classifier w/ Imputer + Standard Scaler:
	Starting cross validation
	Finished cross validation - mean Log Loss Binary: 0.097
	H

We can view more information about the stacking ensemble pipeline (which was the best performing pipeline) by calling `.describe()`.

In [25]:
automl_with_ensembling.best_pipeline.describe()


*******************************************************
* Elastic Net Classifier w/ Imputer + Standard Scaler *
*******************************************************

Problem Type: binary
Model Family: Linear
Number of features: 30

Pipeline Steps
1. Imputer
	 * categorical_impute_strategy : most_frequent
	 * numeric_impute_strategy : median
	 * categorical_fill_value : None
	 * numeric_fill_value : None
2. Standard Scaler
3. Elastic Net Classifier
	 * penalty : elasticnet
	 * C : 8.123565600467177
	 * l1_ratio : 0.47997717237505744
	 * n_jobs : -1
	 * multi_class : auto
	 * solver : saga


## Access raw results

The `AutoMLSearch` class records detailed results information under the `results` field, including information about the cross-validation scoring and parameters.

In [26]:
automl.results

{'pipeline_results': {0: {'id': 0,
   'pipeline_name': 'Mode Baseline Binary Classification Pipeline',
   'pipeline_class': evalml.pipelines.binary_classification_pipeline.BinaryClassificationPipeline,
   'pipeline_summary': 'Baseline Classifier',
   'parameters': {'Baseline Classifier': {'strategy': 'mode'}},
   'mean_cv_score': 3.970423187263591,
   'standard_deviation_cv_score': 0.26606000431837074,
   'high_variance_cv': False,
   'training_time': 0.8083591461181641,
   'cv_data': [{'all_objective_scores': OrderedDict([('Log Loss Binary',
                   4.124033002377396),
                  ('MCC Binary', 0.0),
                  ('Gini', 0.0),
                  ('AUC', 0.5),
                  ('Precision', 0.0),
                  ('F1', 0.0),
                  ('Balanced Accuracy Binary', 0.5),
                  ('Accuracy Binary', 0.8805970149253731),
                  ('# Training', 133),
                  ('# Validation', 67)]),
     'mean_cv_score': 4.124033002377396,
     

## Parallel AutoML

By default, all pipelines in an AutoML batch are evaluated in series.  Pipelines can be evaluated in parallel to improve performance during AutoML search.  This is accomplished by a futures style submission and evaluation of pipelines in a batch.  As of this writing, the pipelines use a threaded model for concurrent evaluation.  This is similar to the currently implemented `n_jobs` parameter in the estimators, which uses increased numbers of threads to train and evaluate estimators.

### Parallelism with Concurrent Futures

The `EngineBase` class is robust and extensible enough to support futures-like implementations from a variety of libraries.  The `CFEngine` extends the `EngineBase` to use the native Python concurrent.futures library.  The `CFEngine` supports both thread- and process-level parallelism.  The type of parallelism can be chosen using either the `ThreadPoolExecutor` or the `ProcessPoolExecutor`.  If either executor is passed a `max_workers` parameter, it will set the number of processes and threads spawned.  If not, the default number of processes will be equal to the number of processors available and the number of threads set to five times the number of processors available.

Note: the cell demonstrating process-level parallelism is commented out due to incompatibility with our ReadTheDocs build.  It can be run successfully locally.

In [27]:
from concurrent.futures import ThreadPoolExecutor

from evalml.automl.engine import CFEngine
from evalml.automl.engine.cf_engine import CFClient

# Use thread-level paralellism
threaded_cf_engine = CFEngine(CFClient(ThreadPoolExecutor()))

automl_cf_threaded = AutoMLSearch(X_train=X, y_train=y,
                      problem_type="binary",
                      allowed_model_families=[ModelFamily.LINEAR_MODEL],
                      engine = threaded_cf_engine)
automl_cf_threaded.search(show_iteration_plot = False)

Using default limit of max_batches=1.

Generating pipelines to search over...
2 pipelines ready for search.

*****************************
* Beginning pipeline search *
*****************************

Optimizing for Log Loss Binary. 
Lower score is better.

Using CFEngine to train and score pipelines.
Searching up to 1 batches for a total of 3 pipelines. 
Allowed model families: linear_model

Evaluating Baseline Pipeline: Mode Baseline Binary Classification Pipeline
Mode Baseline Binary Classification Pipeline:
	Starting cross validation
	Finished cross validation - mean Log Loss Binary: 12.868

*****************************
* Evaluating Batch Number 1 *
*****************************

Elastic Net Classifier w/ Imputer + Standard Scaler:
	Starting cross validation
	Finished cross validation - mean Log Loss Binary: 0.077
Logistic Regression Classifier w/ Imputer + Standard Scaler:
	Starting cross validation
	Finished cross validation - mean Log Loss Binary: 0.077
	High coefficient of vari

In [28]:
# from concurrent.futures import ProcessPoolExecutor

# # Repeat the process but using process-level parallelism
# process_cf_engine = CFEngine(CFClient(ProcessPoolExecutor()))

# automl_cf_process = AutoMLSearch(X_train=X, y_train=y,
#                       problem_type="binary",
#                       allowed_model_families=[ModelFamily.LINEAR_MODEL],
#                       engine = process_cf_engine)
# automl_cf_process.search(show_iteration_plot = False)

### Parallelism with Dask

Thread or process level parallelism can be explicitly evoked for the DaskEngine (as well as the CFEngine).  The `processes` can be set to `True` and the number of processes set using `n_workers`.  If `processes` is set to `False`, then the resulting parallelism will be threaded and `n_workers` will represent the threads used.  Examples of both will follow.

In [29]:
from dask.distributed import Client, LocalCluster

from evalml.automl.engine import DaskEngine

dask_engine_t2 = DaskEngine(Client(LocalCluster(processes=True, n_workers = 2)))
automl_dask_t2 = AutoMLSearch(X_train=X, y_train=y,
                      problem_type="binary",
                      allowed_model_families=[ModelFamily.LINEAR_MODEL],
                      engine = dask_engine_t2)
automl_dask_t2.search(show_iteration_plot = False)

/usr/local/Caskroom/miniconda/base/envs/evalml_dev/lib/python3.9/site-packages/distributed/node.py:160: UserWarning:

Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 49984 instead



Using default limit of max_batches=1.

Generating pipelines to search over...
2 pipelines ready for search.

*****************************
* Beginning pipeline search *
*****************************

Optimizing for Log Loss Binary. 
Lower score is better.

Using DaskEngine to train and score pipelines.
Searching up to 1 batches for a total of 3 pipelines. 
Allowed model families: linear_model

Evaluating Baseline Pipeline: Mode Baseline Binary Classification Pipeline
Mode Baseline Binary Classification Pipeline:
	Starting cross validation
	Finished cross validation - mean Log Loss Binary: 12.868

*****************************
* Evaluating Batch Number 1 *
*****************************

Elastic Net Classifier w/ Imputer + Standard Scaler:
	Starting cross validation
	Finished cross validation - mean Log Loss Binary: 0.077


Loky-backed parallel loops cannot be called in a multiprocessing, setting n_jobs=1
Loky-backed parallel loops cannot be called in a multiprocessing, setting n_jobs=1
Loky-backed parallel loops cannot be called in a multiprocessing, setting n_jobs=1


Logistic Regression Classifier w/ Imputer + Standard Scaler:
	Starting cross validation
	Finished cross validation - mean Log Loss Binary: 0.077
	High coefficient of variation (cv >= 0.2) within cross validation scores.
	Logistic Regression Classifier w/ Imputer + Standard Scaler may not perform as estimated on unseen data.

Search finished after 00:09            
Best pipeline: Logistic Regression Classifier w/ Imputer + Standard Scaler
Best pipeline Log Loss Binary: 0.076807


Loky-backed parallel loops cannot be called in a multiprocessing, setting n_jobs=1


In [30]:
dask_engine_p4 = DaskEngine(Client(LocalCluster(processes=False, n_workers = 4)))

automl_dask_p4 = AutoMLSearch(X_train=X, y_train=y,
                      problem_type="binary",
                      allowed_model_families=[ModelFamily.LINEAR_MODEL],
                      engine = dask_engine_p4)
automl_dask_p4.search(show_iteration_plot = False)

/usr/local/Caskroom/miniconda/base/envs/evalml_dev/lib/python3.9/site-packages/distributed/node.py:160: UserWarning:

Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 50005 instead



Using default limit of max_batches=1.

Generating pipelines to search over...
2 pipelines ready for search.

*****************************
* Beginning pipeline search *
*****************************

Optimizing for Log Loss Binary. 
Lower score is better.

Using DaskEngine to train and score pipelines.
Searching up to 1 batches for a total of 3 pipelines. 
Allowed model families: linear_model

Evaluating Baseline Pipeline: Mode Baseline Binary Classification Pipeline
Mode Baseline Binary Classification Pipeline:
	Starting cross validation
	Finished cross validation - mean Log Loss Binary: 12.868

*****************************
* Evaluating Batch Number 1 *
*****************************

Elastic Net Classifier w/ Imputer + Standard Scaler:
	Starting cross validation
	Finished cross validation - mean Log Loss Binary: 0.077
Logistic Regression Classifier w/ Imputer + Standard Scaler:
	Starting cross validation
	Finished cross validation - mean Log Loss Binary: 0.077
	High coefficient of va

As we can see, a significant performance gain can result from simply using something other than the default `SequentialEngine`.

In [31]:
print("Sequential search duration: %s" % str(automl.search_duration))
print("Concurrent futures (threaded) search duration: %s" % str(automl_cf_threaded.search_duration))
# print("Concurrent futures (process) search duration: %s" % str(automl_cf_process.search_duration))
print("Dask (two threads) search duration: %s" % str(automl_dask_t2.search_duration))
print("Dask (four processes)search duration: %s" % str(automl_dask_p4.search_duration))

Sequential search duration: 19.669402837753296
Concurrent futures (threaded) search duration: 3.979705810546875
Dask (two threads) search duration: 9.009714126586914
Dask (four processes)search duration: 4.219135046005249
